In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

DATA_PATH = "data.json"

In [2]:
# load data
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
        
    #convert lists to numpy arrays
    inputs = np.array(data["mfcc"])
    targets = np.array(data["labels"])
    
    return inputs, targets

In [3]:
# create, train, validation and test sets

# this is changed from CNN in the following ways:
# 1) 
# 2) removed last dimension of 

def prepare_datasets(test_size, validation_size):
    # load data 
    X, y = load_data(DATA_PATH)
    
    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    
    # create train/validation split
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    return X_train, X_validation, X_test, y_train, y_validation, y_test

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2) # test size, validation size (second is out of remainder from first partition)

In [6]:
# build RNN/LSTM networks
def build_model(input_shape):
    # create model
    model = keras.Sequential()
    
    # RNN/LSTM  numkernels, kernal size, activation
    model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(64))
    model.add(keras.layers.Dense(64, activation="relu"))
    # mitigate overfitting
    model.add(keras.layers.Dropout(0.3))
    
    #ouput
    model.add(keras.layers.Dense(10, activation="softmax")) # 10 genres -- get best fit...
    
    return model
              
    
input_shape = (X_train.shape[1], X_train.shape[2]) # (130, 13) in this case.
model = build_model(input_shape)

In [7]:
# compile the network
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer, 
             loss="sparse_categorical_crossentropy",
             metrics=["accuracy"])

In [8]:
# train the CNN 
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

Epoch 1/30
188/188 [==============================] - 34s 182ms/step - loss: 2.2038 - accuracy: 0.2166 - val_loss: 2.0555 - val_accuracy: 0.3020
Epoch 2/30
188/188 [==============================] - 31s 164ms/step - loss: 1.9307 - accuracy: 0.3198 - val_loss: 1.7573 - val_accuracy: 0.3647
Epoch 3/30
188/188 [==============================] - 30s 158ms/step - loss: 1.7279 - accuracy: 0.3910 - val_loss: 1.6071 - val_accuracy: 0.4333
Epoch 4/30
188/188 [==============================] - 30s 157ms/step - loss: 1.6029 - accuracy: 0.4270 - val_loss: 1.5505 - val_accuracy: 0.4393
Epoch 5/30
188/188 [==============================] - 29s 156ms/step - loss: 1.5401 - accuracy: 0.4456 - val_loss: 1.4807 - val_accuracy: 0.4620
Epoch 6/30
188/188 [==============================] - 29s 156ms/step - loss: 1.4801 - accuracy: 0.4709 - val_loss: 1.4224 - val_accuracy: 0.4920
Epoch 7/30
188/188 [==============================] - 30s 157ms/step - loss: 1.4460 - accuracy: 0.4884 - val_loss: 1.4006 - val_ac

In [7]:
# evaluate CNN on the test set
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("accuracy on test set is {}".format(test_accuracy))

79/79 [==============================] - 1s 10ms/step - loss: 0.8386 - accuracy: 0.7091
accuracy on test set is 0.709083616733551


In [10]:
# make prediction on a sample
def predict(model, X, y):
    # X is 3-dim (13, 13, 1) BUT needs an extra dim at the beginning to specify num samples we want to predict
    X = X[np.newaxis, ...]
    
    # prediction [[softmax results]] ... so... 
    prediction = model.predict(X) 
    
    #extract max index
    predicted_index = np.argmax(prediction, axis=1) # yeilds [3], e.g.
    
    print("expected index: {}, predicted: {}".format(y, predicted_index))
    return predicted_index[0]

expected index: 5, predicted: [5]


In [21]:
X = X_test[500]
y = y_test[500]

index = predict(model, X, y)

expected index: 8, predicted: [8]


In [17]:
with open(DATA_PATH, "r") as fp:
    data = json.load(fp)

In [22]:
data["mapping"][index]

'country'

NameError: name 'pt_musicutils' is not defined